In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
dataset = load_dataset("cifar10")

In [3]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.randint(0, 90, (1, )) < 10:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return (image, label)

In [4]:
train_dataset = CatsDataset()

In [5]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

In [20]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [21]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(16, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(256, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [30]:
for epoch in range(40):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.08546325124230734
Epoch 2 Loss: 0.0905677037681364
Epoch 3 Loss: 0.08466553075867854
Epoch 4 Loss: 0.08698857262445862
Epoch 5 Loss: 0.0809807807539299
Epoch 6 Loss: 0.10677010981235535
Epoch 7 Loss: 0.07039279087333922
Epoch 8 Loss: 0.05092078838260121
Epoch 9 Loss: 0.0458751126299286
Epoch 10 Loss: 0.05058152155037139
Epoch 11 Loss: 0.11802765488577117
Epoch 12 Loss: 0.05394208921226346
Epoch 13 Loss: 0.0399569153524698
Epoch 14 Loss: 0.037466223467331215
Epoch 15 Loss: 0.044273987062465235
Epoch 16 Loss: 0.042824513000098006
Epoch 17 Loss: 0.03897625056635232
Epoch 18 Loss: 0.0548521034822912
Epoch 19 Loss: 0.04234361176966292
Epoch 20 Loss: 0.056562273592575436
Epoch 21 Loss: 0.05186741817060407
Epoch 22 Loss: 0.03988926506239423
Epoch 23 Loss: 0.019361005897283744
Epoch 24 Loss: 0.014086676659181145
Epoch 25 Loss: 0.01069903280734304
Epoch 26 Loss: 0.008616469359280435
Epoch 27 Loss: 0.006607977297100695
Epoch 28 Loss: 0.005613669568441428
Epoch 29 Loss: 0.00573869

In [24]:
test_dataset = CatsDataset(train=False)

In [25]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [31]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7426, device='mps:0')
